# LLM Prueba – Colab (distilgpt2)

Este notebook automatiza instalación, preparación de datos, entrenamiento (con reanudación), generación de ejemplos, subida del modelo al Hugging Face Hub y despliegue de una demo en Spaces (Gradio).

Requisitos manuales:
- Cuenta de Hugging Face y un token con permiso `write`.
- (Opcional) Repositorio GitHub `llm-prueba` para clonar el código.

Consejos de recursos (Colab Free):
- Usa `epochs=1`, `block_size=128`, `batch_size=2`, `grad_accum_steps=8`.
- Si se desconecta, vuelve a ejecutar y usa `--resume` para reanudar desde el último checkpoint.

In [ ]:
#@title 1) Configurar entorno y montar Google Drive (opcional)
import os, sys

IN_COLAB = 'google.colab' in sys.modules
BASE_DIR = os.getcwd()

if IN_COLAB:
    try:
        from google.colab import drive  # type: ignore
        drive.mount('/content/drive')
        BASE_DIR = '/content/drive/MyDrive/llm-prueba'
        os.makedirs(BASE_DIR, exist_ok=True)
        %cd $BASE_DIR
        print('Base dir:', BASE_DIR)
    except Exception as e:
        print('No se pudo montar Drive, se usará almacenamiento efímero del runtime.', e)
else:
    print('Ejecutando fuera de Colab. Base dir:', BASE_DIR)


In [ ]:
#@title 2) Instalar dependencias y configurar cachés
import os

# Opcional: cache en Drive para datasets/modelos
if 'BASE_DIR' in globals():
    os.environ['HF_HOME'] = os.path.join(BASE_DIR, 'hf_cache')
    os.environ['TRANSFORMERS_CACHE'] = os.path.join(BASE_DIR, 'hf_cache')
    os.environ['HF_DATASETS_CACHE'] = os.path.join(BASE_DIR, 'hf_cache')

try:
    import torch
    print('Torch:', torch.__version__, 'CUDA:', torch.cuda.is_available())
except Exception as e:
    print('Torch no disponible aún:', e)

# Instalar desde requirements.txt si existe; si no, usa lista fija
if os.path.exists('requirements.txt'):
    !pip -q install -U pip
    !pip -q install -r requirements.txt
else:
    !pip -q install -U pip
    !pip -q install transformers==4.41.2 datasets==2.19.0 accelerate==0.30.1 huggingface_hub==0.23.4 evaluate==0.4.1 gradio==4.36.1 tensorboard==2.17.0 safetensors==0.4.2 tqdm==4.66.4

import torch
print('Torch listo:', torch.__version__, 'CUDA:', torch.cuda.is_available())

In [ ]:
#@title 3) Verificar que el proyecto está presente en BASE_DIR
import os, glob

expected = [
    'src/train.py',
    'src/data/prepare_dataset.py',
    'src/generate.py',
]
missing = [p for p in expected if not os.path.exists(p)]

if missing:
    print('Faltan archivos del proyecto:', missing)
    print('\nAcción manual necesaria:')
    print('- Copia la carpeta completa `llm-prueba` a', os.getcwd(), 'en tu Google Drive (MyDrive).')
    print('- Alternativa: git clone tu repo privado/público en esta carpeta y vuelve a ejecutar esta celda.')
else:
    print('Proyecto OK. Archivos clave presentes.')
    print('Directorio actual:', os.getcwd())
    print('Contenido src/:', glob.glob('src/*'))

In [ ]:
#@title 4) Preparar dataset (opus_books ES por defecto) { display-mode: "form" }
SOURCE = "hf_opus_books" #@param ["hf_opus_books", "local"]
MAX_EXAMPLES = 30000 #@param {type:"integer"}
OUTPUT = "data/dataset.txt" #@param {type:"string"}

import os
os.makedirs('data', exist_ok=True)
os.makedirs('data/raw', exist_ok=True)

if SOURCE == 'local':
    print('Usando archivos locales en data/raw/*.txt')
!python -u src/data/prepare_dataset.py --source $SOURCE --max_examples $MAX_EXAMPLES --output $OUTPUT

import os
print('Tamaño dataset.txt:', os.path.getsize(OUTPUT), 'bytes')
!python - << 'PY'
path = 'data/dataset.txt'
with open(path, 'r', encoding='utf-8') as f:
    for i, line in enumerate(f):
        if i > 10:
            break
        print(line.strip())
PY

In [ ]:
#@title 5) Entrenar modelo (con reanudación opcional) { display-mode: "form" }
EPOCHS = 1 #@param {type:"number"}
BLOCK_SIZE = 128 #@param {type:"integer"}
BATCH_SIZE = 2 #@param {type:"integer"}
GRAD_ACCUM = 8 #@param {type:"integer"}
SAVE_STEPS = 200 #@param {type:"integer"}
LOGGING_STEPS = 50 #@param {type:"integer"}
LEARNING_RATE = 5e-5 #@param {type:"number"}
RESUME = True #@param {type:"boolean"}

import subprocess, sys

cmd = [
    sys.executable, "-u", "src/train.py",
    "--model_name", "distilgpt2",
    "--dataset_path", "data/dataset.txt",
    "--checkpoint_dir", "models/checkpoints",
    "--final_model_dir", "models/final_model",
    "--epochs", str(EPOCHS),
    "--block_size", str(BLOCK_SIZE),
    "--batch_size", str(BATCH_SIZE),
    "--grad_accum_steps", str(GRAD_ACCUM),
    "--learning_rate", str(LEARNING_RATE),
    "--save_steps", str(SAVE_STEPS),
    "--logging_steps", str(LOGGING_STEPS),
    "--eval_ratio", "0.05",
]
if RESUME:
    cmd.append("--resume")

print('Running:', ' '.join(cmd))
subprocess.run(cmd, check=False)


In [ ]:
#@title 6) (Opcional) Ver métricas en TensorBoard
%load_ext tensorboard
%tensorboard --logdir logs --port 6006

In [ ]:
#@title 7) Generar 3 ejemplos de texto con el modelo entrenado
!python -u src/generate.py --model_dir models/final_model --output outputs/examples.txt

import os
print('Ejemplos guardados en outputs/examples.txt, tamaño:', os.path.getsize('outputs/examples.txt'), 'bytes')
!head -n 80 outputs/examples.txt || (Get-Content -Path outputs/examples.txt -TotalCount 80)

In [ ]:
#@title 8) Configurar credenciales HF y nombres de repos { display-mode: "form" }
HF_TOKEN = "" #@param {type:"string"}
HF_USERNAME = "" #@param {type:"string"}
MODEL_REPO_NAME = "llm-prueba-distilgpt2-es" #@param {type:"string"}
SPACE_REPO_NAME = "llm-prueba-demo" #@param {type:"string"}

import os
if not HF_TOKEN or not HF_USERNAME:
    print("[ACCION MANUAL] Define HF_TOKEN y HF_USERNAME en esta celda y vuelve a ejecutarla. Obtén el token en https://huggingface.co/settings/tokens con permiso write.")
else:
    os.environ['hf_token'] = HF_TOKEN
    MODEL_REPO = f"{HF_USERNAME}/{MODEL_REPO_NAME}"
    SPACE_REPO = f"{HF_USERNAME}/{SPACE_REPO_NAME}"
    print('Repos configurados:')
    print('MODEL_REPO =', MODEL_REPO)
    print('SPACE_REPO =', SPACE_REPO)

In [ ]:
#@title 9) Subir modelo a Hugging Face Hub
import os, sys, subprocess

if 'MODEL_REPO' not in globals():
    raise SystemExit('Define HF_TOKEN y HF_USERNAME en la celda 8 para construir MODEL_REPO y SPACE_REPO, y vuelve a ejecutar.')

if not os.environ.get('HF_TOKEN'):
    raise SystemExit('HF_TOKEN no definido. Vuelve a ejecutar la celda 8 con tu token.')

if not os.path.exists('models/final_model/config.json'):
    raise SystemExit('Modelo final no encontrado en models/final_model. Ejecuta el entrenamiento (celda 5).')

cmd = [
    sys.executable, '-u', 'src/upload_to_hub.py',
    '--model_dir', 'models/final_model',
    '--repo_id', MODEL_REPO,
]
print('Subiendo a:', MODEL_REPO)
subprocess.run(cmd, check=False)

In [ ]:
#@title 10) Desplegar Space de Gradio en Hugging Face { display-mode: "form" }
import os, sys, subprocess

if 'SPACE_REPO' not in globals() or 'MODEL_REPO' not in globals():
    raise SystemExit('Define HF_TOKEN y HF_USERNAME en la celda 8 para construir MODEL_REPO y SPACE_REPO, y vuelve a ejecutar.')

if not os.environ.get('HF_TOKEN'):
    raise SystemExit('HF_TOKEN no definido. Vuelve a ejecutar la celda 8 con tu token.')

cmd = [
    sys.executable, '-u', 'deploy/deploy_space.py',
    '--space_repo', SPACE_REPO,
    '--model_repo', MODEL_REPO,
]
print('Desplegando Space:', SPACE_REPO)
subprocess.run(cmd, check=False)

print('\nIMPORTANTE: Ve a Settings del Space y define la variable de entorno MODEL_ID con', MODEL_REPO)